In [46]:
import sys
sys.path.append('E:\weather\Lib\site-packages')
# 打印环境变量
# print(sys.path)

In [4]:
import os
import time
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt


# 求各年份下数据各channel的均值和方差
def get_mean_std():
    if not os.path.exists('data_analysis'):
        os.makedirs('data_analysis')
    data_complete = xr.Dataset()
    for year in range(2007,2012):
        os.makedirs(f'data_analysis/{year}') if not os.path.exists(f'data_analysis/{year}') else None
        data = xr.open_zarr(f'./data/weather_round1_train_{year}', consolidated=True).x
        print(data.shape)
        t0 = time.time()
        data_mean = data.mean(dim=['time'])
        t1 = time.time()
        data_std = data.std(dim=['time'])
        t2 = time.time()
        data_mean.name = f'{year}_mean'
        data_std.name = f'{year}_std'
        data_temp = xr.merge([data_mean, data_std])
        data_temp.to_zarr(f'data_analysis/{year}/{year}_mean_std.zarr', mode='w')
        data_complete = xr.merge([data_complete, data_temp])
        print(f'year:{year} mean time:{t1-t0} std time:{t2-t1}')
    data_complete.to_zarr(f'data_analysis/2007_2011_mean_std.zarr', mode='w')
    return data_complete

# 保存torch.tensor到pt


data_complete = get_mean_std()

(1460, 70, 161, 161)
year:2007 mean time:20.991554737091064 std time:15.509047985076904
(1464, 70, 161, 161)
year:2008 mean time:20.918192148208618 std time:8.187369346618652
(1460, 70, 161, 161)
year:2009 mean time:16.05082678794861 std time:8.268658876419067
(1460, 70, 161, 161)
year:2010 mean time:21.70847773551941 std time:28.432596683502197
(1460, 70, 161, 161)
year:2011 mean time:16.967739582061768 std time:15.771782159805298


In [25]:
print(data_complete, '\n'*2)

data_mean, data_std = [], []
for year in range(2007,2012):
    data_mean.append(data_complete[f'{year}_mean'])
    data_std.append(data_complete[f'{year}_std'])
data_mean = xr.concat(data_mean, dim='time')
data_mean.name = 'total_mean'
data_mean.coords['time'] = [2007,2008,2009,2010,2011]
data_std = xr.concat(data_std, dim='time')
data_std.name = 'total_std'
data_std.coords['time'] = [2007,2008,2009,2010,2011]
data_mean = data_mean.mean(dim=['time'])
data_std = data_std.mean(dim=['time'])
data_complete = xr.merge([data_complete, data_mean, data_std])
print(data_complete)
data_complete.to_zarr('data_analysis/2007_2011_mean_std.zarr', mode='w')

<xarray.Dataset>
Dimensions:     (channel: 70, lat: 161, lon: 161)
Coordinates:
  * channel     (channel) <U5 'z50' 'z100' 'z150' 'z200' ... 'v10' 'msl' 'tp'
  * lat         (lat) float32 50.0 49.75 49.5 49.25 ... 10.75 10.5 10.25 10.0
  * lon         (lon) float32 100.0 100.2 100.5 100.8 ... 139.5 139.8 140.0
Data variables:
    2007_mean   (channel, lat, lon) float32 -0.7675 -0.7698 ... 0.7268 0.7093
    2007_std    (channel, lat, lon) float32 2.041 2.041 2.041 ... 1.256 1.248
    2008_mean   (channel, lat, lon) float32 -0.9976 -1.001 ... 0.6535 0.6492
    2008_std    (channel, lat, lon) float32 2.245 2.247 2.249 ... 1.199 1.203
    2009_mean   (channel, lat, lon) float32 -0.9303 -0.9335 ... 0.7194 0.7089
    2009_std    (channel, lat, lon) float32 2.221 2.224 2.226 ... 1.287 1.281
    2010_mean   (channel, lat, lon) float32 -0.8981 -0.9019 ... 0.5497 0.5492
    2010_std    (channel, lat, lon) float32 2.475 2.478 2.481 ... 1.172 1.168
    2011_mean   (channel, lat, lon) float32 -0.94

In [2]:
import pandas as pd
data = xr.open_zarr('data_analysis/2007_2011_mean_std.zarr')

# 将数据按经度和维度展开成一维，计算各channel之间的相关性
data_mean = data['total_mean'].values.transpose(1,2,0).reshape(-1,70)
print(data_mean.shape, '\n', data.channel.values)
data_std = data['total_std'].values.transpose(1,2,0).reshape(-1,70)
# 计算相关系数，并给出变量名，保留两位小数
corr = pd.DataFrame(np.corrcoef(data_mean, rowvar=False), columns=data.channel.values, index=data.channel.values).round(2)
corr_std = pd.DataFrame(np.corrcoef(data_std, rowvar=False), columns=data.channel.values, index=data.channel.values).round(2)
print(corr.shape, corr_std.shape)
corr.to_csv('data_analysis/2007_2011_mean_corr.csv')
corr_std.to_csv('data_analysis/2007_2011_std_corr.csv')


NameError: name 'xr' is not defined

In [17]:
corr = pd.read_csv('data_analysis/2007_2011_mean_corr.csv', index_col=0)

# 找出均值相关性绝对值大于0.9的变量对，并打印相关性值
for target in ['t2m', 'u10', 'v10', 'msl', 'tp']:
    threshold = 0.8
    print(f'{target}相关性大于{threshold}的变量有：')
    # print(corr[corr[target].abs()>threshold][target], '\n')
    print(len(corr[corr[target].abs()>threshold][target].index))
    print(corr[corr[target].abs()>threshold][target].index, '\n'*2)

t2m相关性大于0.8的变量有：
28
Index(['z100', 'z150', 'z200', 'z250', 'z300', 'z400', 'z500', 'z600', 'z700',
       't50', 't100', 't150', 't250', 't300', 't400', 't500', 't600', 't700',
       't850', 't925', 't1000', 'u50', 'v500', 'r50', 'r100', 'r150', 'r200',
       't2m'],
      dtype='object') 


u10相关性大于0.8的变量有：
4
Index(['u850', 'u925', 'u1000', 'u10'], dtype='object') 


v10相关性大于0.8的变量有：
2
Index(['v1000', 'v10'], dtype='object') 


msl相关性大于0.8的变量有：
14
Index(['z1000', 'u50', 'u100', 'u150', 'u200', 'u250', 'u300', 'u400', 'r50',
       'r100', 'r150', 'r200', 'msl', 'tp'],
      dtype='object') 


tp相关性大于0.8的变量有：
16
Index(['t100', 't150', 't250', 't300', 't400', 't500', 'u50', 'r50', 'r100',
       'r150', 'r200', 'r850', 'r925', 'r1000', 'msl', 'tp'],
      dtype='object') 




In [ ]:
"""
t2m相关性大于0.92的变量有：
t100   -0.93
t150   -0.94
t300    0.93
t400    0.93
t2m     1.00

u10相关性大于0.8的变量有：
u850     0.85
u925     0.94
u1000    1.00
u10      1.00

v10相关性大于0.75的变量有：
v1000    0.98
v10      1.00

msl相关性大于0.85的变量有：
z1000    0.99
u100     0.92
u150     0.87
r100    -0.88
r150    -0.88
r200    -0.87
msl      1.00

tp相关性大于0.85的变量有：
u50    -0.87
r150    0.88
r200    0.87
r925    0.89
tp      1.00

"""

In [75]:
target_features = {'t2m': ['t100', 't150', 't300', 't400', 't2m'],
                   'u10': ['u850', 'u925', 'u1000', 'u10'],
                   'v10': ['v1000', 'v10'],
                   'msl': ['z1000', 'u100', 'u150', 'r100', 'r150', 'r200', 'msl'],
                   'tp': ['u50', 'r150', 'r200', 'r925', 'tp']}

data = xr.open_zarr('data/weather_round1_train_2007', consolidated=True).x
# print(data, '\n'*2)
# 打印数据中目标变量对应的channel的特征变量
for target in ['t2m', 'u10', 'v10', 'msl', 'tp']:
    print(f'{target}对应的特征变量有：')
    print(data.sel(channel=target_features[target]).shape, '\n'*2)
    # print(data[target_features[target]].channel.values, '\n')


t2m对应的特征变量有：
(1460, 5, 161, 161) 


u10对应的特征变量有：
(1460, 4, 161, 161) 


v10对应的特征变量有：
(1460, 2, 161, 161) 


msl对应的特征变量有：
(1460, 7, 161, 161) 


tp对应的特征变量有：
(1460, 5, 161, 161) 


